Loading Dataset

In [ ]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/Positive_tensors.zip
!unzip -q Positive_tensors.zip

In [ ]:
! wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/Negative_tensors.zip
!unzip -q Negative_tensors.zip

Importing Libraries


In [ ]:

import torchvision.models as models
from PIL import Image
from torchvision import transforms
import torch.nn as nn
import time
import torch
import matplotlib.pylab as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import h5py
from matplotlib.pyplot import imshow
import matplotlib.pylab as plt
import pandas as pd
import os
torch.manual_seed(0)

Dataset Class


In [ ]:


class Dataset(Dataset):

    # Constructor
    def __init__(self,transform=None,train=True):

        directory = "/content"
        positive="Positive_tensors"
        negative='Negative_tensors'

        positive_file_path=os.path.join(directory,positive)
        negative_file_path=os.path.join(directory,negative)
        positive_files=[os.path.join(positive_file_path,file) for file in os.listdir(positive_file_path) if file.endswith(".pt")]
        negative_files=[os.path.join(negative_file_path,file) for file in os.listdir(negative_file_path) if file.endswith(".pt")]
        number_of_samples=len(positive_files)+len(negative_files)

        self.all_files=[None]*number_of_samples
        self.all_files[::2]=positive_files
        self.all_files[1::2]=negative_files
        self.transform = transform
        #torch.LongTensor
        self.Y=torch.zeros([number_of_samples]).type(torch.LongTensor)
        self.Y[::2]=1
        self.Y[1::2]=0

        if train:
            self.all_files=self.all_files[0:30000]
            self.Y=self.Y[0:30000]
            self.len=len(self.all_files)
        else:
            self.all_files=self.all_files[30000:]
            self.Y=self.Y[30000:]
            self.len=len(self.all_files)

    # Get the length
    def __len__(self):
        return self.len

    # Getter
    def __getitem__(self, idx):

        image=torch.load(self.all_files[idx])
        y=self.Y[idx]


        if self.transform:
            image = self.transform(image)

        return image, y

print("done")

Splitting and Transforming Dataset 


In [ ]:
mean=[0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
# Type your code here
composed = transforms.Compose([transforms.Resize(224), transforms.Normalize(mean, std)])
train_dataset = Dataset(transform=composed, train=True)
validation_dataset = Dataset(transform=composed, train=False)
print("done")

Setup GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Load the Model

In [ ]:
# Load the pre-trained model resnet18
model= models.resnet18(pretrained=True)


In [11]:
# Set the parameter cannot be trained for the pre-trained model
for param in model.parameters():
    param.requires_grad=False


Define the Output Layer

In [12]:
model.fc=nn.Linear(512,7)

In [ ]:
#  model initialization
model.to(device)

Dataloader


In [16]:
train_loader = DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=100, shuffle=False)


Optimizer & Criterion


In [17]:
optimizer = torch.optim.Adam([parameters  for parameters in model.parameters() if parameters.requires_grad],lr=0.001)
criterion = nn.CrossEntropyLoss()

Training the Model

In [ ]:
n_epochs=1
loss_list=[]
accuracy_list=[]
correct=0
N_test=len(validation_dataset)
N_train=len(train_dataset)
start_time = time.time()

Loss=0
start_time = time.time()
for epoch in range(n_epochs):
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        model.train()
        #clear gradient
        optimizer.zero_grad()
        #make a prediction
        z = model(x)
        # calculate loss
        loss = criterion(z, y)
        # calculate gradients of parameters
        loss.backward()
        # update parameters
        optimizer.step()
        loss_list.append(loss.data)

    with torch.no_grad():
        for x_test, y_test in validation_loader:
            # set model to eval
            x_test, y_test = x_test.to(device), y_test.to(device)
            model.eval()
            #make a prediction
            z = model(x_test)
            #find max
            _, yhat = torch.max(z.data, 1)

            #Calculate misclassified  samples in mini-batch
            #hint +=(yhat==y_test).sum().item()
            correct += (yhat == y_test).sum().item()

    accuracy=correct/N_test
    accuracy_list.append(accuracy)


Validation Accuracy

In [ ]:
print("the accuracy of the model is: ", accuracy)

Plotting the Loss

In [ ]:
loss_list_cpu = [loss.cpu().detach().numpy() for loss in loss_list]

# Plot the loss
plt.plot(loss_list_cpu)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

Misclassified Samples


In [ ]:
count = 0
for i, (x, y) in enumerate(torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=1)):

    x, y = x.to(device), y.to(device)
    model.eval()
    z = model(x)
    _, yhat = torch.max(z, 1)

    if yhat != y:
        print(f"Sample {i} predicted value: {yhat}  actual value: {y}")
        count += 1

    if count >= 4:
        break
